In [1]:
import pandas as pd
import re
import os
import numpy as np
import json


In [230]:
case_summaries_df = pd.read_json("case_summaries.json", orient=None, typ='frame', dtype=True, 
                 convert_axes=True, convert_dates=True, keep_default_dates=True, 
                 numpy=False, precise_float=False, date_unit=None, encoding=None, lines=False)

In [5]:
# need to git clone https://github.com/walkerdb/supreme_court_transcripts repo and cd to the "oyez" folder

load_path = "./cases/"
save_path = "./cases/"

# Rename all files by removing periods from file names because who puts periods in file names?
for file_name in os.listdir(load_path):
    if re.search("\.(?!json)", file_name):
        file_load_path = os.path.join(load_path, file_name)    
        file_rename = re.sub("\.(?!json)", "_", file_name)
        
        file_save_path = os.path.join(save_path, file_rename)
        os.rename(file_load_path, file_save_path)
        print file_name, "\t\t\trenamed\t", file_rename



In [112]:
transcript_list = []
counter = 0
transcript_dict = {}

for file_name in os.listdir(save_path):
    if counter < 10000:
        if re.search("t0", file_name):
            #print file_name
            file_path = os.path.join(save_path, file_name)
            with open(file_path) as f:
                #data = json.load(f)

                transcript_name = "OA" + re.sub("\.json", "", file_name)
                transcript_name = re.sub("-", "_", transcript_name)
                transcript_list.append((counter, file_name, transcript_name))
                print counter, transcript_name           
    #             transcript_name = json.load(f)
                transcript_dict[transcript_name] = json.load(f)['transcript']
                print "DONE!!"
                #transcript_name = pd.read_json("/cases/1974_73_1595.json", orient="split", typ='frame', dtype=True, convert_axes=True, convert_dates=True, keep_default_dates=True, numpy=False, precise_float=False, date_unit=None, encoding=None, lines=True)


                counter +=1
        

0 OA1989_89_386_t01
DONE!!
1 OA1968_47_t01
DONE!!
2 OA1972_71_1422_t01
DONE!!
3 OA1984_84_5004_t01
DONE!!
4 OA1992_92_311_t01
DONE!!
5 OA1965_280_t01
DONE!!
6 OA1984_83_1416_t01
DONE!!
7 OA1974_73_1121_t01
DONE!!
8 OA1957_106_t01
DONE!!
9 OA1982_82_11_t01
DONE!!
10 OA2002_01_7662_t01
DONE!!
11 OA1993_93_5418_t01
DONE!!
12 OA2012_11_10362_t01
DONE!!
13 OA1978_78_91_t01
DONE!!
14 OA1962_27_t02
DONE!!
15 OA1986_86_234_t01
DONE!!
16 OA1985_84_902_t01
DONE!!
17 OA1969_528_t01
DONE!!
18 OA1966_637_t02
DONE!!
19 OA1957_50_t01
DONE!!
20 OA1960_126_t01
DONE!!
21 OA1956_183_t02
DONE!!
22 OA1994_93_6892_t01
DONE!!
23 OA1981_81_328_t01
DONE!!
24 OA2002_01_1184_t01
DONE!!
25 OA1980_80_901_t01
DONE!!
26 OA1977_77_334_t01
DONE!!
27 OA1985_84_1667_t01
DONE!!
28 OA1979_79_253_t01
DONE!!
29 OA1993_92_9093_t01
DONE!!
30 OA1961_31_t01
DONE!!
31 OA1976_76_60_t01
DONE!!
32 OA1995_95_340_t01
DONE!!
33 OA1989_89_65_t01
DONE!!
34 OA1980_79_1056_t01
DONE!!
35 OA1984_83_1097_t01
DONE!!
36 OA1963_88_t01
DONE!!
37

In [177]:
# c = 0
# for x in range(len(transcript_dict["OA2016_15_214_t01"]["sections"])):
#     print "section:", c
#     c +=1
#     if transcript_dict["OA2016_15_214_t01"]["sections"][x]['turns'][0]['speaker']['roles'] is not None:
#         print transcript_dict["OA2016_15_214_t01"]["sections"][x]['turns'][0]
#     else:
#         print "no speaker role"
# print transcript_dict["OA1996_96_270_t01"].keys()
# print transcript_dict["OA1996_96_270_t01"]['sections'][0]['turns'][7]['speaker']['roles'] is None

# print transcript_dict["OA1965_59_t01"]["sections"][2]['turns'][9]['speaker'], "\n"
# print transcript_dict["OA1965_59_t01"]["sections"][2]['turns'][9]['text_blocks']

section: 0
{u'text_blocks': [{u'stop': 7.06, u'text': u'We will hear argument first this morning in case 15-214, Murr v. Wisconsin. Mr. Groen.', u'byte_stop': 0, u'byte_start': 0, u'start': 0}], u'stop': 7.06, u'byte_start': 0, u'start': 0, u'byte_stop': 0, u'speaker': {u'view_count': 106, u'last_name': u'Roberts', u'name': u'John G. Roberts, Jr.', u'roles': [{u'role_title': u'Chief Justice of the United States', u'date_end': 0, u'date_start': 1127970000, u'institution_name': u'Supreme Court of the United States', u'appointing_president': u'George W. Bush', u'href': u'https://api.oyez.org/preson_role/scotus_justice/2730', u'type': u'scotus_justice', u'id': 2730}], u'thumbnail': {u'href': u'https://api.oyez.org/sites/default/files/images/people/john_g_roberts_jr/john_g_roberts_jr.thumb.png', u'mime': u'image/png', u'id': 32683, u'size': 52305}, u'href': u'https://api.oyez.org/people/john_g_roberts_jr', u'length_of_service': 3842, u'identifier': u'john_g_roberts_jr', u'ID': 15086}}
secti

In [ ]:
# for line in range(len(transcript_dict["OA1965_59_t01"]['sections'][0]['turns'])):
#     print transcript_dict["OA1965_59_t01"]['sections'][1]['turns'][line], "\n"

In [114]:
# pd.DataFrame(transcript_dict["OA1965_59_t01"]['sections'][0]['turns'])
transcript_list[8000][2]

'OA1958_45_t02'

In [275]:
             #################### This code literally takes days to run #######################

turns_columns = ['transcript_id', 'title', 'speaker', 'speaker_ID', 'speaker_role', 
                 'speaker_appointing_pres', 'text_start', 'text_stop', 'text']

turns_df = pd.DataFrame(columns = turns_columns)
counter = 0
# errors = []
# processed = []
# for t_id in [(0,0,'OA2016_15_214_t01')]:
for t_id in transcript_list[2321:]:
    transcript_id = t_id[2]
    print transcript_id
    
    if transcript_id not in processed and transcript_dict[transcript_id]:
        transcript = transcript_dict[transcript_id]
    else:
        print "Error with this transcript, appears to be empty file"
        errors.append(transcript_id)
    # Go through every section of the transcript
    for sect in transcript['sections']:
        # Go through every text turn, extract speaker_ID, speaker, and role/appointing president if any
#         print sect
        for turn in range(len(sect['turns'])):
            turns_dict = {}
            speaker = sect['turns'][turn]['speaker']
#             print "turn:", turn
            if speaker:
#                 print "speaker_ID:", speaker['ID']
                turns_dict['speaker_ID'] = speaker['ID']

#                 print "speaker:", speaker['name']
                turns_dict['speaker'] = speaker['name']

                if speaker['roles'] and speaker['roles'] is not None:
#                     print "speaker_role:", speaker['roles'][0]['type']
                    turns_dict['speaker_role'] = speaker['roles'][0]['type']

#                     print "speaker_appointing_pres:", speaker['roles'][0]['appointing_president'], "\n"
                    turns_dict['speaker_appointing_pres'] = speaker['roles'][0]['appointing_president']
                else:
                    turns_dict['speaker_role'] = "not_a_justice"
                    turns_dict['speaker_appointing_pres'] = "NA"
            else:
                pass
#                 print "speaker unidentified"
            # Extract starting time of turn
#             print "text_start:", sect['turns'][turn]['text_blocks'][0]['start']
            turns_dict['text_start'] = sect['turns'][turn]['text_blocks'][0]['start']

#             print "\ntext:"

            # Turns are divided into text blocks, go through every text block in a turn and extract text
            turns_dict['text'] = ""
            for text_block in sect['turns'][turn]['text_blocks']:
    #             print "text_block_start:", text_block['start'], "\ntext_block_stop:", text_block['stop']
#                 print text_block['text']
                turns_dict['text'] += text_block['text'] + " "

            # Extract stopping time of turn
#             print "\ntext_stop:", text_block['stop']
            turns_dict['text_stop'] = text_block['stop']
            turns_dict['title'] = transcript['title']
            turns_dict['transcript_id'] = re.sub("OA", "", transcript_id)

#             print turns_dict
            turns_df = turns_df.append(pd.Series(turns_dict), ignore_index=True)
            processed.append(transcript_id)
    print "\t", counter
    counter += 1
    if counter in [1, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000]:
        turns_df.to_csv('all_turns.csv', encoding='utf-8', mode = 'a')
        print "Appended", counter, "transcripts!!!!!!"



OA1961_3_t01
	0
Appended 1 transcripts!!!!!!
OA1959_7_t01
	1
OA1957_95_t01
	2
OA1985_85_198_t01
Error with this transcript, appears to be empty file
	3
OA1976_76_333_t01
	4
OA1969_13_t02
	5
OA1957_127_t01
	6
OA1980_79_5780_t01
	7
OA1969_14_t01
	8
OA1986_86_231_t01
	9
OA1990_89_1679_t01
	10
OA1991_90_889_t01
	11
OA1991_90_1361_t01
	12
OA1970_109_t01
	13
OA1976_75_1053_t01
	14
OA1999_98_1701_t01
	15
OA1957_46_t02
	16
OA1991_90_1038_t02
	17
OA1968_131_t01
	18
OA1977_76_1836_t01
	19
OA1959_76_t01
	20
OA1981_80_1614_t01
	21
OA2006_06_5618_t01
	22
OA1973_72_1713_t01
	23
OA1977_76_5856_t01
	24
OA1958_23_t02
	25
OA1958_12_t02
	26
OA1968_646_t01
	27
OA1965_219_t01
	28
OA1961_554_t01
	29
OA1962_142_t02
	30
OA1976_75_699_t01
	31
OA1969_829_t02
	32
OA1975_74_1044_t01
	33
OA1959_11_t01
	34
OA2003_02_1624_t01
	35
OA1981_80_2043_t01
	36
OA1979_78_756_t01
	37
OA2003_02_1196_t01
	38
OA1965_45_t01
	39
OA1983_82_766_t01
	40
OA1988_86_1856_t01
	41
OA1991_91_538_t01
	42
OA2008_07_1216_t01
	43
OA1966_101_t0

KeyboardInterrupt: 

In [282]:
turns_df['text_duration'] = turns_df['text_stop'] - turns_df['text_start']
# turns_df.to_csv('turns_part2.csv', encoding='utf-8')
turns_df


,transcript_id,title,speaker,speaker_ID,speaker_role,speaker_appointing_pres,text_start,text_stop,text,text_duration
0,1961_3_t01,Metlakatla Indian Community v. Egan,Earl Warren,15099.0,scotus_justice,Dwight D. Eisenhower,0.000,18.872,"Metlakatla Indian Community, Annette Islands R...",18.872
1,1961_3_t01,Metlakatla Indian Community v. Egan,Richard Schifter,40513.0,not_a_justice,NA,18.872,160.048,"Mr. Chief Justice, may it please the Court. Th...",141.176
2,1961_3_t01,Metlakatla Indian Community v. Egan,Earl Warren,15099.0,scotus_justice,Dwight D. Eisenhower,160.048,170.010,"May I ask you, how about the invitational was ...",9.962
3,1961_3_t01,Metlakatla Indian Community v. Egan,Richard Schifter,40513.0,not_a_justice,NA,170.010,173.906,It was originally -- I'm not certain -- first ...,3.896
4,1961_3_t01,Metlakatla Indian Community v. Egan,Earl Warren,15099.0,scotus_justice,Dwight D. Eisenhower,173.906,174.514,It was not by--,0.608
5,1961_3_t01,Metlakatla Indian Community v. Egan,Richard Schifter,40513.0,not_a_justice,NA,174.514,284.503,It was not by an act of Congress. These people...,109.989
6,1961_3_t01,Metlakatla Indian Community v. Egan,Earl Warren,15099.0,scotus_justice,Dwight D. Eisenhower,284.503,286.785,May I interrupt just once more please?,2.282
7,1961_3_t01,Metlakatla Indian Community v. Egan,Richard Schifter,40513.0,not_a_justice,NA,286.785,289.396,Yes sir.,2.611
8,1961_3_t01,Metlakatla Indian Community v. Egan,Earl Warren,15099.0,scotus_justice,Dwight D. Eisenhower,289.396,296.671,My question is prompted by the fact you said t...,7.275
9,1961_3_t01,Metlakatla Indian Community v. Egan,Richard Schifter,40513.0,not_a_justice,NA,296.671,296.954,Yes.,0.283


In [289]:
speaker_df = pd.DataFrame({'text_joined': turns_df.groupby(['transcript_id', 'speaker_ID','speaker','speaker_role'])['text'].apply(lambda x: ''.join(x))})
# speaker_df = speaker_df.reset_index()
# speaker_df.to_csv('speakers_part2.csv', encoding = 'utf-8')
speaker_df


text_joined
transcript_id   speaker_ID speaker              speaker_role                                                     
1955_102_t02    15037.0    Sherman Minton       scotus_justice  Well, they did consider these other things alo...
                15050.0    Felix Frankfurter    scotus_justice  Is that the whole order? Is that the order? Is...
                15063.0    Stanley Reed         scotus_justice  Did it require a special equipment to carry th...
                15089.0    Hugo L. Black        scotus_justice  (Voice Overlap) is -- is it your idea that if ...
                15099.0    Earl Warren          scotus_justice  Number 102, United States of America and Inter...
                15107.0    William O. Douglas   scotus_justice  (Voice Overlap) Suppose it often showed the or...
                18345.0    Charles F. Barber    not_a_justice   May it please the Court. This case is here on ...
                29301.0    Jusitce Burton       not_a_justice   Did the -- Did the District Court find that yo...
                38608.0    Robert N. Burchmore  not_a_justice   Mr. Chief Justice, Your Honors, may it please ...
1955_10_t02     15050.0    Felix Frankfurter    scotus_justice                      What is that you're reading? 
                15099.0    Earl Warren          scotus_justice  Number 10, Commonwealth of Pennsylvania versus...
                24711.0    Frank F. Truscott    not_a_justice   If the Court pleases, this case comes before Y...
1955_110_t01    15050.0    Felix Frankfurter    scotus_justice  Before you -- before you move over to that, I'...
                15089.0    Hugo L. Black        scotus_justice  What judge did that? You said that difficulty ...
                15099.0    Earl Warren          scotus_justice  I think, Mr. Ford, your time has -- -- expired...
                18927.0    Charles E. Ford      not_a_justice   -- of this Court, the Dennis case, the one -- ...
                26179.0    Gray Thoron          not_a_justice   May it please the Court. Petitioner's argument...
1955_110_t02    15037.0    Sherman Minton       scotus_justice  Is there a practice authorizing the federal co...
                15063.0    Stanley Reed         scotus_justice  When -- when did he send this? I -- I understa...
                15099.0    Earl Warren          scotus_justice  Number 110, Harold I. Cammer versus United Sta...
                18927.0    Charles E. Ford      not_a_justice   May it please this Honorable Court. The statut...
1955_112_t01    15099.0    Earl Warren          scotus_justice  Number 112, Amos Reece versus the State of Geo...
                20730.0    Daniel Duke          not_a_justice   This is Daniel Duke appearing for the petition...
1955_117_t01    15099.0    Earl Warren          scotus_justice  -- versus Union Pacific Railroad, et al, Washi...
                24128.0    Frank E. Holman      not_a_justice   Mr. Chief Justice and Members of the Court. I ...
1955_129_t01    15050.0    Felix Frankfurter    scotus_justice  Suppose the Supreme Court of Illinois should, ...
                15063.0    Stanley Reed         scotus_justice  The conclusion of law has been made. They made...
                15083.0    Tom C. Clark         scotus_justice                           (Inaudible) (Inaudible) 
                15089.0    Hugo L. Black        scotus_justice  Are there any facts that would have to be adde...
                15099.0    Earl Warren          scotus_justice  Number 129, George W. Doud et al., versus Orvi...
...                                                                                                           ...
2016_15_927_t01 42213.0    Seth P. Waxman       not_a_justice   Mr. Chief Justice, and may it please the Court...
                62762.0    Martin J. Black      not_a_justice   Mr. Chief Justice, and may it please the Court...
2016_16_142_t01 15068.0    Samuel A. Alito, Jr. scotus_justice  Well, how would this work -- how would your ru..

In [196]:
# role_df = pd.DataFrame({'text_joined': turns_df.groupby(['transcript_id','speaker_role'])['text'].apply(lambda x: ''.join(x))})
# role_df = speaker_df.reset_index()
# role_df.to_csv('roles_part2.csv', encoding = 'utf-8')

In [284]:
duration_df = pd.DataFrame(turns_df.groupby(['transcript_id', 'speaker_ID','speaker','speaker_role'])['text_duration'].sum())
# duration_df = duration_df.reset_index()
# duration_df.to_csv('durations_part2.csv', encoding = 'utf-8')
duration_df

text_duration
transcript_id   speaker_ID speaker              speaker_role                 
1955_102_t02    15037.0    Sherman Minton       scotus_justice          8.303
                15050.0    Felix Frankfurter    scotus_justice        605.925
                15063.0    Stanley Reed         scotus_justice         80.908
                15089.0    Hugo L. Black        scotus_justice         80.839
                15099.0    Earl Warren          scotus_justice         48.258
                15107.0    William O. Douglas   scotus_justice         49.883
                18345.0    Charles F. Barber    not_a_justice        1387.219
                29301.0    Jusitce Burton       not_a_justice           5.465
                38608.0    Robert N. Burchmore  not_a_justice        1082.326
1955_10_t02     15050.0    Felix Frankfurter    scotus_justice          1.138
                15099.0    Earl Warren          scotus_justice         40.339
                24711.0    Frank F. Truscott    not_a_justice        2725.257
1955_110_t01    15050.0    Felix Frankfurter    scotus_justice        381.333
                15089.0    Hugo L. Black        scotus_justice        117.012
                15099.0    Earl Warren          scotus_justice        126.032
                18927.0    Charles E. Ford      not_a_justice        1335.004
                26179.0    Gray Thoron          not_a_justice        1685.637
1955_110_t02    15037.0    Sherman Minton       scotus_justice          8.949
                15063.0    Stanley Reed         scotus_justice          8.174
                15099.0    Earl Warren          scotus_justice         17.782
                18927.0    Charles E. Ford      not_a_justice         602.822
1955_112_t01    15099.0    Earl Warren          scotus_justice        104.590
                20730.0    Daniel Duke          not_a_justice        2467.528
1955_117_t01    15099.0    Earl Warren          scotus_justice         37.241
                24128.0    Frank E. Holman      not_a_justice         441.473
1955_129_t01    15050.0    Felix Frankfurter    scotus_justice        723.135
                15063.0    Stanley Reed         scotus_justice         32.021
                15083.0    Tom C. Clark         scotus_justice          6.075
                15089.0    Hugo L. Black        scotus_justice         58.344
                15099.0    Earl Warren          scotus_justice         75.382
...                                                                       ...
2016_15_927_t01 42213.0    Seth P. Waxman       not_a_justice        1271.780
                62762.0    Martin J. Black      not_a_justice        1201.119
2016_16_142_t01 15068.0    Samuel A. Alito, Jr. scotus_justice         95.300
                15084.0    Ruth Bader Ginsburg  scotus_justice         97.700
                15086.0    John G. Roberts, Jr. scotus_justice      -3171.245
                15094.0    Elena Kagan          scotus_justice        197.285
                15113.0    Anthony M. Kennedy   scotus_justice         46.435
                15131.0    Sonia Sotomayor      scotus_justice        158.670
                15139.0    Stephen G. Breyer    scotus_justice        335.030
                56123.0    Brian H. Fletcher    not_a_justice         901.466
                62644.0    Adam G. Unikowsky    not_a_justice        1339.359
2016_16_327_t01 15068.0    Samuel A. Alito, Jr. scotus_justice        267.060
                15084.0    Ruth Bader Ginsburg  scotus_justice        145.710
                15086.0    John G. Roberts, Jr. scotus_justice      -3413.030
                15094.0    Elena Kagan          scotus_justice        366.765
                15113.0    Anthony M. Kennedy   scotus_justice        216.330
                15131.0    Sonia Sotomayor      scotus_justice        432.740
                15139.0    Stephen G. Breyer    scotus_justice        385.195
                22289.0    Eric J. Feigin       not_a_justice         849.360
        

In [274]:

# for t_id in transcript_list:

#     transcript_id = t_id[2]
#     if transcript_id not in processed and transcript_dict[transcript_id]:
#         print transcript_id, " Not processed yet"

#     else:
#         print transcript_id, "Processed!!!"

In [273]:
# co = 0
# for t_id in transcript_list:

#     transcript_id = t_id[2]
#     print transcript_id[:2234]
#     if transcript_id == 'OA1961_3_t01':
#         print "FOUND IT", co
#         break
#     co+=1

In [288]:
transcript_list[4734]


(4734, '1964_50-t02.json', 'OA1964_50_t02')

In [293]:
case_summaries_df

,ID,citation,description,docket_number,href,justia_url,name,question,term,timeline,view_count
0,49051,"{u'volume': u'382', u'href': u'https://api.oye...",None,510,https://api.oyez.org/cases/1966/510,https://supreme.justia.com/cases/federal/us/38...,"American Trucking Assns., Inc. v. United States",None,1966,"[{u'dates': [-124826400], u'href': u'https://a...",0
1,49422,"{u'volume': u'387', u'href': u'https://api.oye...",None,1043,https://api.oyez.org/cases/1966/1043,https://supreme.justia.com/cases/federal/us/38...,Gills v. California,None,1966,"[{u'dates': [-81284400], u'href': u'https://ap...",0
2,49891,"{u'volume': u'391', u'href': u'https://api.oye...",None,1269,https://api.oyez.org/cases/1967/1269,https://supreme.justia.com/cases/federal/us/39...,Central Bank & Trust Co. v. United States,None,1967,"[{u'dates': [-50439600], u'href': u'https://ap...",0
3,49934,"{u'volume': u'391', u'href': u'https://api.oye...",None,373,https://api.oyez.org/cases/1967/373,https://supreme.justia.com/cases/federal/us/39...,California v. Phillips Petroleum Co.,None,1967,"[{u'dates': [-51044400], u'href': u'https://ap...",0
4,50231,"{u'volume': u'396', u'href': u'https://api.oye...",None,343,https://api.oyez.org/cases/1969/343,https://supreme.justia.com/cases/federal/us/39...,Boston & Maine R. Co. v. United States,None,1969,"[{u'dates': [-4471200], u'href': u'https://api...",0
5,50299,"{u'volume': u'394', u'href': u'https://api.oye...",None,1047,https://api.oyez.org/cases/1968/1047,https://supreme.justia.com/cases/federal/us/39...,Chicago v. United States,None,1968,"[{u'dates': [-22010400], u'href': u'https://ap...",0
6,50599,"{u'volume': u'405', u'href': u'https://api.oye...",None,70-161,https://api.oyez.org/cases/1971/70-161,https://supreme.justia.com/cases/federal/us/40...,Richardson v. Wright,,1971,"[{u'dates': [64130400], u'href': u'https://api...",0
7,50600,"{u'volume': u'405', u'href': u'https://api.oye...",None,70-5058,https://api.oyez.org/cases/1971/70-5058,https://supreme.justia.com/cases/federal/us/40...,Lynch v. Household Finance Corporation,,1971,"[{u'dates': [60933600], u'href': u'https://api...",0
8,50601,"{u'volume': u'405', u'href': u'https://api.oye...",None,70-5004,https://api.oyez.org/cases/1971/70-5004,https://supreme.justia.com/cases/federal/us/40...,Humphrey v. Cady,,1971,"[{u'dates': [60933600], u'href': u'https://api...",0
9,50602,"{u'volume': u'407', u'href': u'https://api.oye...",None,71-6425,https://api.oyez.org/cases/1971/71-6425,https://supreme.justia.com/cases/federal/us/40...,Ivan V. v. City of New York,,1971,"[{u'dates': [77173200], u'href': u'https://api...",0
